# Bahdanau Attention

:label:`sec_seq2seq_attention`

When we encountered machine translation in :numref:`sec_seq2seq`,
we designed an encoder-decoder architecture 
based on two RNNs for sequence-to-sequence learning.
Specifically, the RNN encoder transforms a variable-length sequence
into a fixed-shape context variable.
Then, the RNN decoder generates the output (target) sequence token by token
based on the generated tokens and the context variable.
However, while not all input (source) tokens
are relevant when decoding a particular target token,
the *same* context variable
that encodes the entire input sequence
is still used at each decoding step.

In a separate but related challenge 
of handwriting generation for a given text sequence,
:citet:`Graves.2013` designed a differentiable attention model
to align text characters with the much longer pen trace,
where the alignment moves only in one direction.
Inspired by the idea of learning to align,
:citet:`Bahdanau.Cho.Bengio.2014` proposed a differentiable attention model
without the severe unidirectional alignment limitation.
When predicting a token,
if not all the input tokens are relevant,
the model aligns (or attends)
only to parts of the input sequence 
that are deemed relevant to the current prediction.
This is achieved by constructing the context variable
via an attention mechanism. 

## Model

To describe the  Bahdanau-style attention mechanism 
for the RNN encoder-decoder below,
we follow the same notation as in :numref:`sec_seq2seq`.
The new attention-based model follows 
the sequence-to-sequence architecture of :numref:`sec_seq2seq`,
but with the context variable $\mathbf{c}$ in :eqref:`eq_seq2seq_s_t`
replaced by $\mathbf{c}_{t'}$
at any decoding time step $t'$.
Suppose that there are $T$ tokens in the input sequence,
the context variable at the decoding time step $t'$
is the output of attention pooling:

$$\mathbf{c}_{t'} = \sum_{t=1}^T \alpha(\mathbf{s}_{t' - 1}, \mathbf{h}_t) \mathbf{h}_t,$$

where the decoder hidden state
$\mathbf{s}_{t' - 1}$ at time step $t' - 1$ is the query,
and the encoder hidden states $\mathbf{h}_t$
are both the keys and values,
and the attention weight $\alpha$
is computed as in
:eqref:`eq_attn-scoring-alpha`
using the additive attention scoring function
defined by :eqref:`eq_additive-attn`.

This RNN encoder-decoder architecture
augmented  Bahdanau attention 
is depicted in :numref:`fig_s2s_attention_details`.

![Layers in an RNN encoder-decoder model with Bahdanau attention.](https://d2l.ai/_images/seq2seq-attention-details.svg)
:label:`fig_s2s_attention_details`

In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.Vocabulary
import ai.djl.modality.nlp.embedding.TrainableWordEmbedding
import jp.live.ugai.d2j.timemachine.RNNModelScratch
import jp.live.ugai.d2j.timemachine.TimeMachine.trainCh8
import jp.live.ugai.d2j.timemachine.TimeMachineDataset
import jp.live.ugai.d2j.timemachine.Vocab
import jp.live.ugai.d2j.RNNModel
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Accumulator
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.util.TrainingChapter9
import jp.live.ugai.d2j.lstm.Decoder
import jp.live.ugai.d2j.lstm.Encoder
import jp.live.ugai.d2j.lstm.EncoderDecoder
import jp.live.ugai.d2j.util.NMT
import jp.live.ugai.d2j.attention.MultiHeadAttention
import jp.live.ugai.d2j.attention.AdditiveAttention
import jp.live.ugai.d2j.PositionalEncoding
import java.util.Locale
import kotlin.random.Random
import kotlin.collections.List
import kotlin.collections.Map
import kotlin.Pair

## Defining the Decoder with Attention

To implement the RNN encoder-decoder 
with Bahdanau attention,
we only need to redefine the decoder.
To visualize the learned attention weights more conveniently,
the following `AttentionDecoder` class
defines [**the base interface for
decoders with attention mechanisms**].

In [2]:
abstract class AttentionDecoder : AbstractBlock() {
    var attentionWeights: NDArray? = null
    abstract fun initState(encOutputs: NDList, encValidLens: NDArray): NDList
    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        throw UnsupportedOperationException("Not implemented")
    }
}

Now let's [**implement
the RNN decoder with Bahdanau attention**]
in the following `Seq2SeqAttentionDecoder` class.
The state of the decoder is initialized with
(i) the encoder final-layer hidden states at all the time steps 
(as keys and values of the attention);
(ii) the encoder all-layer hidden state at the final time step 
(to initialize the hidden state of the decoder);
and (iii) the encoder valid length 
(to exclude the padding tokens in attention pooling).
At each decoding time step,
the decoder final-layer hidden state at the previous time step 
is used as the query of the attention.
As a result, both the attention output
and the input embedding are concatenated
as input of the RNN decoder.

In [3]:
class Seq2SeqAttentionDecoder(vocabSize: Long, embedSize: Int, numHiddens: Int, numLayers: Int,
                 dropout: Float =0f) : AttentionDecoder() {
    val attention = AdditiveAttention(numHiddens, dropout)
    val embedding : TrainableWordEmbedding
    val rnn = GRU.builder()
            .setNumLayers(numLayers)
            .setStateSize(numHiddens)
            .optReturnState(true)
            .optBatchFirst(false)
            .optDropRate(dropout)
            .build()
    val linear = Linear.builder().setUnits(vocabSize).build() 
    
    init {
        val list: List<String> = (0 until vocabSize).map { it.toString() }
        val vocab: Vocabulary = DefaultVocabulary(list)
        // Embedding layer
        embedding = TrainableWordEmbedding.builder()
            .optNumEmbeddings(vocabSize.toInt())
            .setEmbeddingSize(embedSize)
            .setVocabulary(vocab)
            .build()
//        addChildBlock("embedding", embedding)
    }
    
    override fun initState(encOutputs: NDList, encValidLens: NDArray) : NDList {
        val outputs = encOutputs[0]
        val hiddenState = encOutputs[1]
        return NDList(outputs.swapAxes(0,1), hiddenState, encValidLens)
    }
    
    override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        val encOutputs = params!![0] as NDArray
        val hiddenState = params!![1] as NDArray
        val encValidLens = params!![2] as NDArray
        var X = inputs[0]
        // The output `X` shape: (`batchSize`, `numSteps`, `embedSize`)
        X = embedding.forward(ps, NDList(X), training, params)[0].swapAxes(0,1)
        for(x in 0 until X.size(0)) {
            val query = hiddenState[-1].expandDims(1)
            val context = attention.forward(ps, NDList(query, encOutputs, encOutputs, encValidLens), training)
            val xArray = context[0].concat(X[x].expandDims(1), 1)
            val out = rnn.forward(ps, NDList(xArray.swapAxes(0,1)), false ,PairList(listOf(ai.djl.util.Pair("hidden_state", hiddenState))))
            
        }
        // In RNN models, the first axis corresponds to time steps
        X = X.swapAxes(0, 1)
        return rnn.forward(ps, NDList(X), training)
    }
}

/*
def forward(self, X, state):
        # Shape of enc_outputs: (batch_size, num_steps, num_hiddens).
        # Shape of hidden_state: (num_layers, batch_size, num_hiddens)
        enc_outputs, hidden_state, enc_valid_lens = state
        # Shape of the output X: (num_steps, batch_size, embed_size)
        X = self.embedding(X).swapaxes(0, 1)
        outputs, self._attention_weights = [], []
        for x in X:
            # Shape of query: (batch_size, 1, num_hiddens)
            query = np.expand_dims(hidden_state[-1], axis=1)
            # Shape of context: (batch_size, 1, num_hiddens)
            context = self.attention(
                query, enc_outputs, enc_outputs, enc_valid_lens)
            # Concatenate on the feature dimension
            x = np.concatenate((context, np.expand_dims(x, axis=1)), axis=-1)
            # Reshape x as (1, batch_size, embed_size + num_hiddens)
            out, hidden_state = self.rnn(x.swapaxes(0, 1), hidden_state)
            hidden_state = hidden_state[0]
            outputs.append(out)
            self._attention_weights.append(self.attention.attention_weights)
        # After fully connected layer transformation, shape of outputs:
        # (num_steps, batch_size, vocab_size)
        outputs = self.dense(np.concatenate(outputs, axis=0))
        return outputs.swapaxes(0, 1), [enc_outputs, hidden_state,
                                        enc_valid_lens]
*/

Line_10.jupyter-kts (46:111 - 112) Unexpected tokens (use ';' to separate expressions on the same line)

In the following, we [**test the implemented
decoder**] with Bahdanau attention
using a minibatch of 4 sequence inputs
of 7 time steps.

## [**Training**]

Similar to :numref:`sec_seq2seq_training`,
here we specify hyperparameters,
instantiate
an encoder and a decoder with Bahdanau attention,
and train this model for machine translation.


After the model is trained,
we use it to [**translate a few English sentences**]
into French and compute their BLEU scores.

By [**visualizing the attention weights**]
when translating the last English sentence,
we can see that each query assigns non-uniform weights
over key-value pairs.
It shows that at each decoding step,
different parts of the input sequences
are selectively aggregated in the attention pooling.

## Summary

When predicting a token, if not all the input tokens are relevant, the RNN encoder-decoder with Bahdanau attention selectively aggregates different parts of the input sequence. This is achieved by treating the context variable as an output of additive attention pooling.
In the RNN encoder-decoder, Bahdanau attention treats the decoder hidden state at the previous time step as the query, and the encoder hidden states at all the time steps as both the keys and values.

## Exercises

1. Replace GRU with LSTM in the experiment.
1. Modify the experiment to replace the additive attention scoring function with the scaled dot-product. How does it influence the training efficiency?